<a href="https://colab.research.google.com/github/finiteautomata/pytorch-sentiment-analysis/blob/master/7_Finetuning_BERT_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6 - Transformers for Sentiment Analysis

In this notebook we will be using the transformer model, first introduced in [this](https://arxiv.org/abs/1706.03762) paper. Specifically, we will be using the BERT (Bidirectional Encoder Representations from Transformers) model from [this](https://arxiv.org/abs/1810.04805) paper. 

Transformer models are considerably larger than anything else covered in these tutorials. As such we are going to use the [transformers library](https://github.com/huggingface/transformers) to get pre-trained transformers and use them as our embedding layers. We will freeze (not train) the transformer and only train the remainder of the model which learns from the representations produced by the transformer. In this case we will be using a multi-layer bi-directional GRU, however any model can learn from these representations.

## Preparing Data

First, as always, let's set the random seeds for deterministic results.

In [0]:
%%capture
!pip install transformers

In [0]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

The transformer has already been trained with a specific vocabulary, which means we need to train with the exact same vocabulary and also tokenize our data in the same way that the transformer did when it was initially trained.

Luckily, the transformers library has tokenizers for each of the transformer models provided. In this case we are using the BERT model which ignores casing (i.e. will lower case every word). We get this by loading the pre-trained `bert-base-uncased` tokenizer.

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

The `tokenizer` has a `vocab` attribute which contains the actual vocabulary we will be using. We can check how many tokens are in it by checking its length.

We can get the indexes of the special tokens by converting them using the vocabulary...

In [0]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)


...or by explicitly getting them from the tokenizer.

In [10]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


Another thing we need to handle is that the model was trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the `max_model_input_sizes` for the version of the transformer we want to use. In this case, it is 512 tokens.

In [11]:
max_input_length = 256 #tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

256


Previously we have used the `spaCy` tokenizer to tokenize our examples. However we now need to define a function that we will pass to our `TEXT` field that will handle all the tokenization for us. It will also cut down the number of tokens to a maximum length. Note that our maximum length is 2 less than the actual maximum length. This is because we need to append two tokens to each sequence, one to the start and one to the end.

In [0]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [13]:
tokenizer.tokenize("hello mother fucker how are you fucking doing? ")

['hello',
 'mother',
 'fuck',
 '##er',
 'how',
 'are',
 'you',
 'fucking',
 'doing',
 '?']

Now we define our fields. The transformer expects the batch dimension to be first, so we set `batch_first = True`. As we already have the vocabulary for our text, provided by the transformer we set `use_vocab = False` to tell torchtext that we'll be handling the vocabulary side of things. We pass our `tokenize_and_cut` function as the tokenizer. The `preprocessing` argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. Finally, we define the special tokens - making note that we are defining them to be their index value and not their string value, i.e. `100` instead of `[UNK]` This is because the sequences will already be converted into indexes.

We define the label field as before.

In [0]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)


We load the data and create the validation splits as before.

In [15]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 39.5MB/s]


In [16]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


We can check an example and ensure that the text has already been numericalized.

In [17]:
print(vars(train_data.examples[6]))

{'text': [2023, 2003, 2028, 1997, 1996, 4788, 3572, 1997, 2143, 3298, 2140, 1045, 2031, 2464, 1999, 1037, 2146, 2096, 1012, 2009, 2003, 2061, 9643, 1010, 2008, 1045, 2572, 2025, 2469, 2073, 2000, 4088, 1010, 2030, 2130, 2065, 2009, 2003, 4276, 2009, 1012, 1996, 5436, 2003, 1996, 2613, 3291, 1010, 1998, 1045, 2514, 3374, 2005, 1005, 18230, 1005, 2004, 2002, 8509, 1999, 1037, 11519, 2836, 2005, 2010, 2112, 1012, 2021, 2008, 5436, 1012, 1012, 1012, 2821, 6203, 2821, 6203, 1012, 1045, 3391, 2293, 1996, 2126, 2379, 1996, 2203, 2002, 9020, 2000, 3769, 2013, 1996, 3329, 1997, 1037, 3137, 2000, 1996, 2327, 1010, 5819, 1996, 7739, 2003, 2006, 1996, 2126, 1012, 1037, 7105, 1997, 1037, 2154, 2030, 2048, 3138, 2032, 2035, 1997, 2274, 2781, 999, 1045, 2071, 2175, 2006, 1024, 2021, 2009, 3475, 1005, 1056, 4276, 2009, 1012, 4237, 2013, 1996, 11844, 3098, 1006, 2029, 2130, 1037, 2274, 2095, 2214, 2052, 2022, 2583, 2000, 16014, 1996, 9560, 1997, 1007, 1996, 2717, 2003, 3298, 2140, 1012, 3374, 12455, 10

We can use the `convert_ids_to_tokens` to transform these indexes back into readable tokens.

In [18]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['this', 'is', 'one', 'of', 'the', 'worse', 'cases', 'of', 'film', 'drive', '##l', 'i', 'have', 'seen', 'in', 'a', 'long', 'while', '.', 'it', 'is', 'so', 'awful', ',', 'that', 'i', 'am', 'not', 'sure', 'where', 'to', 'begin', ',', 'or', 'even', 'if', 'it', 'is', 'worth', 'it', '.', 'the', 'plot', 'is', 'the', 'real', 'problem', ',', 'and', 'i', 'feel', 'sorry', 'for', "'", 'sly', "'", 'as', 'he', 'puts', 'in', 'a', 'decent', 'performance', 'for', 'his', 'part', '.', 'but', 'that', 'plot', '.', '.', '.', 'oh', 'dear', 'oh', 'dear', '.', 'i', 'particularly', 'love', 'the', 'way', 'near', 'the', 'end', 'he', 'manages', 'to', 'pop', 'from', 'the', 'foot', 'of', 'a', 'mountain', 'to', 'the', 'top', ',', 'whilst', 'the', 'helicopter', 'is', 'on', 'the', 'way', '.', 'a', 'climb', 'of', 'a', 'day', 'or', 'two', 'takes', 'him', 'all', 'of', 'five', 'minutes', '!', 'i', 'could', 'go', 'on', ':', 'but', 'it', 'isn', "'", 't', 'worth', 'it', '.', 'apart', 'from', 'the', 'grim', 'opening', '(', 'w

Although we've handled the vocabulary for the text, we still need to build the vocabulary for the labels.

In [0]:
LABEL.build_vocab(train_data)

In [20]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f6a86eeee18>, {'neg': 0, 'pos': 1})


As before, we create the iterators. Ideally we want to use the largest batch size that we can as I've found this gives the best results for transformers.

In [0]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Build the Model

Next, we'll load the pre-trained model, making sure to load the same model as we did for the tokenizer.

In [22]:

from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

Next, we'll define our actual model. 

Instead of using an embedding layer to get embeddings for our text, we'll be using the pre-trained transformer model. These embeddings will then be fed into a GRU to produce a prediction for the sentiment of the input sentence. We get the embedding dimension size (called the `hidden_size`) from the transformer via its config attribute. The rest of the initialization is standard.

Within the forward pass, we wrap the transformer in a `no_grad` to ensure no gradients are calculated over this part of the model. The transformer actually returns the embeddings for the whole sequence as well as a *pooled* output. The [documentation](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel) states that the pooled output is "usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence", hence we will not be using it. The rest of the forward pass is the standard implementation of a recurrent model, where we take the hidden state over the final time-step, and pass it through a linear layer to get our predictions.

In [0]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

Next, we create an instance of our model using standard hyperparameters.

In [0]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

We can check how many parameters the model has. Our standard models have under 5M, but this one has 112M! Luckily, 110M of these parameters are from the transformer and we will not be training those.

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


## Train the Model

As is standard, we define our optimizer and criterion (loss function).

In [0]:
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=1e-5)

# Copied from transformers page...please check
N_EPOCHS = 5

num_training_steps = N_EPOCHS * len(train_iterator)
num_warmup_steps = len(train_iterator)
warmup_proportion = float(num_warmup_steps) / float(num_training_steps)  # 0.1

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=num_warmup_steps, 
    num_training_steps=num_training_steps
) 

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

Next, we'll define functions for: calculating accuracy, performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes.

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
from tqdm import tqdm_notebook as tqdm

def train(model, iterator, optimizer, criterion, scheduler):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    pbar = tqdm(iterator, ncols=800)
    for i, batch in enumerate(pbar):
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        lr = optimizer.param_groups[0]["lr"]

        desc = f"Loss {epoch_loss / (i+1):.3f} -- Acc {epoch_acc / (i+1):.3f} -- LR {lr:.5f}"
        pbar.set_description(desc)
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Finally, we'll train our model. This takes considerably longer than any of the previous models due to the size of the transformer. Even though we are not training any of the transformer's parameters we still need to pass the data through the model which takes a considerable amount of time on a standard GPU.

In [30]:
from tqdm.autonotebook import tqdm

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in tqdm(range(N_EPOCHS), ncols=800):
    
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, scheduler)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

	Train Loss: 0.472 | Train Acc: 75.00%
	 Val. Loss: 0.257 |  Val. Acc: 89.82%


	Train Loss: 0.212 | Train Acc: 91.68%
	 Val. Loss: 0.246 |  Val. Acc: 90.45%


	Train Loss: 0.137 | Train Acc: 95.15%
	 Val. Loss: 0.237 |  Val. Acc: 91.48%


	Train Loss: 0.086 | Train Acc: 97.22%
	 Val. Loss: 0.275 |  Val. Acc: 91.46%


	Train Loss: 0.058 | Train Acc: 98.34%
	 Val. Loss: 0.293 |  Val. Acc: 91.52%



We'll load up the parameters that gave us the best validation loss and try these on the test set - which gives us our best results so far!

In [37]:

model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.226 | Test Acc: 91.66%


## Inference

We'll then use the model to test the sentiment of some sequences. We tokenize the input sequence, trim it down to the maximum length, add the special tokens to either side, convert it to a tensor, add a fake batch dimension and then pass it through our model.

In [0]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [39]:
predict_sentiment(model, tokenizer, "This film is terrible")

0.03625558689236641

In [40]:
predict_sentiment(model, tokenizer, "This film is great")

0.9832587242126465

In [43]:
predict_sentiment(model, tokenizer, "To be completely honest, I would recommend everyone to see this movie")

0.8182678818702698